In [ ]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier as RandomForest
import random
from sklearn import linear_model
from datetime import datetime
import gc

pd.set_option("display.max_columns", 2000)


In [ ]:
CHUNK_SIZE = 10000

start = datetime.now()

reader_date = pd.read_csv("train_date.csv",chunksize=CHUNK_SIZE, dtype=np.float32)
reader_num = pd.read_csv("train_numeric.csv",chunksize=CHUNK_SIZE, dtype=np.float32)

#分割して読み込み
loop = 0
cnt_0 = 0
cnt_1 = 0
for df_chunk_date in reader_date:
    df_chunk_num = reader_num.get_chunk(CHUNK_SIZE)
    
    df_chunk = pd.merge(on="Id", left=df_chunk_date, right=df_chunk_num)
    cnt_0 = cnt_0 + len(df_chunk[df_chunk['Response'] == 0].index)
    cnt_1 = cnt_1 + len(df_chunk[df_chunk['Response'] == 1].index)

    if loop == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk])
    
    print("loop : {} / Rows = {}".format(loop, len(df.index)))

    loop = loop + 1

print("0:{}, 1:{}".format(cnt_0, cnt_1))
print("{} -> {}".format(start, datetime.now()))
print("Memory:{}".format(df.memory_usage(index=True).sum()))

In [ ]:
df.head()

In [ ]:
### dateに関して。 Responseが1,0の違いを比較してみる。

In [ ]:
df_date_ok = df.loc[df['Response'] == 0]
df_date_ng = df.loc[df['Response'] == 1]

In [ ]:
df_date_ok.head()

In [ ]:
# OK情報

In [ ]:
print (df_date_ok.info())
print (df_date_ok.count())

In [ ]:
df_date_ok.head()

In [ ]:
CHUNKSIZE = 100000

TRAIN_DATE        = "train_date.csv"
TRAIN_NUMERIC     = "train_numeric.csv"
TRAIN_CATEGORICAL = "train_categorical.csv"
TEST_DATE         = "test_date.csv"
TEST_NUMERIC      = "test_numeric.csv"
TEST_CATEGORICAL  = "test_categorical.csv"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
 

In [ ]:
chunk_train_date = pd.read_csv(TRAIN_DATE, nrows=NROWS, chunksize=CHUNKSIZE)

df_train_date = chunk_train_date.get_chunk(10)

df_train_date.head()

In [ ]:
STATION_NUM = 52

date_cols = df_train_date.columns

date_feature_list = []

for cur_station_id in range(STATION_NUM):
    date_feature_list_one = []
    for column_name in date_cols:
        if column_name != 'Id':
            r = parse("L{}_S{}_D{}", column_name)
            if r is not None:
                col_line_id    = int(r[0])
                col_station_id = int(r[1])
                
                if cur_station_id == col_station_id:
                    date_feature_list_one.append(column_name)

    date_feature_list.append(date_feature_list_one)



In [ ]:
date_feature_list[0]

In [ ]:
stationFeatureNumList = []
#stationFeatureNumList.append(len(date_feature_list[]))
for station in range(STATION_NUM):
    stationFeatureNumList.append(len(date_feature_list[station]))
    
        

In [ ]:
sStation = pd.Series(stationFeatureNumList)
print(sStation.head())
plt.plot(sStation )


In [ ]:
# featureの数に偏りがありそう。-> Station26, 25, 28, 29, 30　 難易度高いのか、何か情報いっぱい出してる。
print(sStation[27])

In [ ]:
df_id  = df_date_ok.loc[:, 'Id']
df_id.head()

In [ ]:
date_feature_list[0]

In [ ]:
print(df_date_ng.columns)


df_date_ng_stations_list = []
df_date_ok_stations_list = []
for i in range(STATION_NUM):
    tmp_list = date_feature_list[i];
    tmp_list.insert(0, ID_COLUMN)

    df_date_ng_stations_list.append(df_date_ng.loc[:, tmp_list])
    df_date_ok_stations_list.append(df_date_ok.loc[:, tmp_list])


    

In [ ]:
df_date_ng_stations_list[0].head()

In [ ]:
df_date_ok_stations_list[3].head()

In [ ]:
station_Used_in_ng = []
for ng in df_date_ng_stations_list:
    station_Used_in_ng.append(ng.drop('Id', axis=1).iloc[:,0])   

plt.scatter(station_Used_in_ng[0].index ,station_Used_in_ng[0], s=1)

In [ ]:
print('station_Used_in_ng[0].fillna(0).values.max()')
print(len(station_Used_in_ng[0]))

In [ ]:
station_Used_in_ok = []
for ok in df_date_ok_stations_list:
    station_Used_in_ok.append(ok.drop('Id', axis=1).iloc[:,0])

plt.scatter(station_Used_in_ok[0].index ,station_Used_in_ok[0], s=1)

In [ ]:
stations_top = []
for st in date_feature_list:
    stations_top.append(st[0])

stations_top

In [ ]:
df_station_top = df.loc[:, stations_top]
df_station_top.head()

In [ ]:
df_station_top_isna = df_station_top.isna()
df_station_top_isna['Response'] = df.Response
df_station_top_isna.head()

In [ ]:
start = datetime.now()
C = df_station_top_isna.corr().abs()
print (C.Response.sort_values(ascending=False))

print("{} -> {}".format(start, datetime.now()))


In [ ]:
## 32,33,34,26 怪しい。数が多いのが原因かもしれないが、ここの通過点や数字には意味がありそう。

In [ ]:
print (date_feature_list[26][0])
print (date_feature_list[32][0])
print (date_feature_list[33][0])
print (date_feature_list[34][0])


In [ ]:
print (df_date_ng.info())
print (df_date_ng.count())

In [ ]:
plt.scatter(station_Used_in_ng[33].index ,station_Used_in_ng[33], s=1)

In [ ]:
plt.scatter(station_Used_in_ok[33].index ,station_Used_in_ok[33], s=1)

#### date_Is_Naの可視化

In [38]:
start = datetime.now()
df_all = df.fillna(10000)
print(df.info())
print("{} -> {}".format(start, datetime.now()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1183747 entries, 0 to 3746
Columns: 1158 entries, Id to Response
dtypes: float32(1158)
memory usage: 5.1 GB
None
2018-07-14 12:01:02.572409 -> 2018-07-14 12:01:06.729331


In [39]:
start = datetime.now()
C = df_all.corr().abs()
C.Response.sort_values(ascending=False)

print("{} -> {}".format(start, datetime.now()))


2018-07-14 12:01:06.744924 -> 2018-07-14 12:48:07.263837


In [40]:
#C.to_csv('output/c_isna_output.csv')
C.to_csv('output/c_date_isna_output.csv')

In [41]:
C.Response.sort_values(ascending=False)
# X = df.loc[C.columns[0:20]]

Response        1.000000
L3_S32_D3852    0.075995
L3_S33_D3874    0.040897
L3_S33_D3872    0.040897
L3_S33_D3870    0.040897
L3_S33_D3868    0.040897
L3_S33_D3866    0.040897
L3_S33_D3864    0.040897
L3_S33_D3862    0.040897
L3_S33_D3858    0.040897
L3_S33_D3856    0.040897
L3_S33_D3860    0.040897
L3_S34_D3883    0.033068
L3_S34_D3881    0.033068
L3_S34_D3879    0.033068
L3_S34_D3877    0.033068
L3_S34_D3875    0.033068
L1_S24_D1536    0.015204
L1_S24_D1522    0.015203
L1_S24_D1527    0.015203
L1_S24_D1550    0.015203
L1_S24_D1532    0.015203
L1_S24_D1541    0.015203
L1_S24_D1546    0.015203
L1_S24_D1554    0.015203
L1_S24_D1579    0.015187
L1_S24_D1576    0.015187
L1_S24_D1570    0.015187
L1_S24_D1572    0.015187
L1_S24_D1574    0.015187
                  ...   
L3_S35_D3895    0.000270
L3_S35_D3897    0.000270
L1_S24_D999     0.000270
L1_S24_D1001    0.000270
L1_S24_D1013    0.000270
L1_S24_D1011    0.000270
L1_S24_D1009    0.000270
L1_S24_D1007    0.000270
L1_S24_D1005    0.000270
